In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
from datetime import date
from datetime import datetime
import unidecode
import re

%matplotlib inline

In [164]:
all_players = pd.read_csv('../DATA/gpt_food.csv')
players_stats = pd.read_csv('../DATA/player_stats.csv')
player_key = pd.read_csv('../DATA/player_key.csv')
#current_squad = pd.read_csv('../ChatGPT/current_squad.csv')

In [165]:
all_players.head(2)

,Unnamed: 0,player_id,fifa_version,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,0,231747,24,k. mbappe,Kylian Mbappé Lottin,"ST, LW",91,94,181500000.0,230000.0,...,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,1,239085,24,e. haaland,Erling Braut Haaland,ST,91,94,185000000.0,340000.0,...,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3


In [166]:
players_stats.head(2)

,Unnamed: 0,rank,player_name,nationality,team,position,overall_rating,pace,shooting,passing,dribbling,defense,physical
0,0,1,Kylian Mbappe,NaN,NaN,ST,91,97,90,80,92,36,78
1,1,2,Alexia Putellas,NaN,NaN,CM,91,82,90,91,92,72,78


In [167]:
player_key['name'] = player_key['name'].str.lower()
player_key['name'] = player_key['name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
player_key.rename(columns = {'name':'player_name'}, inplace = True)
player_key = player_key.drop(columns = ['Unnamed: 0'])
player_key

,id,player_name
0,231747,kylian mbappe
1,239085,erling haaland
2,192985,kevin de bruyne
3,158023,lionel messi
4,165153,karim benzema
...,...,...
15840,266249,cvl remtluanga
15841,269541,wu yuhang
15842,269534,yin jie
15843,272761,lyu jiaqiang


In [168]:
#convert to lower case and drop random unneeded row
players_stats['player_name'] = players_stats['player_name'].str.lower()
#players_stats = players_stats.drop(columns = ['Unnamed: 0'])

#split first and last name
players_stats[['first_name', 'last_name']] = players_stats['player_name'].str.split(' ', 1, expand = True)

#combine into short_name
players_stats['short_name'] = players_stats.apply(lambda row: f"{row['first_name'][0]}. {row['last_name']}", axis = 1)
players_stats

,Unnamed: 0,rank,player_name,nationality,team,position,overall_rating,pace,shooting,passing,dribbling,defense,physical,first_name,last_name,short_name
0,0,1,kylian mbappe,NaN,NaN,ST,91,97,90,80,92,36,78,kylian,mbappe,k. mbappe
1,1,2,alexia putellas,NaN,NaN,CM,91,82,90,91,92,72,78,alexia,putellas,a. putellas
2,2,3,erling haaland,NaN,NaN,ST,91,89,93,66,80,45,88,erling,haaland,e. haaland
3,3,4,kevin de bruyne,NaN,NaN,CM,91,72,88,94,87,65,78,kevin,de bruyne,k. de bruyne
4,4,5,aitana bonmati,NaN,NaN,CM,90,81,84,83,91,75,73,aitana,bonmati,a. bonmati
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17321,17321,17318,deven sawhney,NaN,NaN,LB,47,62,26,34,42,43,57,deven,sawhney,d. sawhney
17322,17322,17318,wang tengda,NaN,NaN,CM,47,61,37,45,48,43,47,wang,tengda,w. tengda
17323,17323,17318,wu yuhang,NaN,NaN,CDM,47,56,34,43,45,43,51,wu,yuhang,w. yuhang
17324,17324,17318,ye daoxin,NaN,NaN,CB,47,58,28,33,39,49,48,ye,daoxin,y. daoxin


In [169]:
#pivoted from creating scratch team because it appears that players are made up
squad = all_players[all_players['club_name'].str.contains('Argyle', na = False)]
squad = squad[['player_id','short_name', 'long_name', 'player_positions', 'value_eur', 'wage_eur']]


## for some reason this didn't work
#test_squad[['first_name', 'last_name']] = test_squad['long_name'].str.split(' ', expand=True).iloc[:, [0,-1]]


squad['first_name'] = test_squad['long_name'].str.split().str[0]
squad['last_name'] = test_squad['long_name'].str.split().str[-1]
squad['first_name'] = test_squad['first_name'].str.lower()
squad['last_name'] = test_squad['last_name'].str.lower()
squad

,player_id,short_name,long_name,player_positions,value_eur,wage_eur,first_name,last_name
3630,240987,m. cooper,Michael John Cooper,GK,3600000.0,7000.0,michael,cooper
3633,243353,b. mumba,Bali Mumba,"LM, LWB, RWB",4200000.0,9000.0,bali,mumba
5347,252066,f. azaz,Finn Isaac Azaz,"RM, LM, CAM",3100000.0,27000.0,finn,azaz
5636,222576,pleguezuelo,Julio José Pleguezuelo Selva,CB,1500000.0,8000.0,julio,selva
5718,237943,d. scarr,Daniel George Scarr,CB,1300000.0,8000.0,daniel,scarr
6040,228366,m. gillesphey,Macaulay Gillesphey,CB,1300000.0,8000.0,macaulay,gillesphey
6790,236474,m. bundu,Mustapha Bundu,"RW, RM",1400000.0,8000.0,mustapha,bundu
6872,192231,j. edwards,Joseph Robert Edwards,"RWB, LWB, RB",825000.0,7000.0,joseph,edwards
8458,253461,l. cundle,Luke James Cundle,"CM, CAM, RW",1900000.0,12000.0,luke,cundle
8500,242423,c. wright,Callum Wright,CM,1900000.0,5000.0,callum,wright


In [170]:
result = pd.merge(players_stats, player_key, on = 'player_name', how = 'inner')
result

,Unnamed: 0,rank,player_name,nationality,team,position,overall_rating,pace,shooting,passing,dribbling,defense,physical,first_name,last_name,short_name,id
0,0,1,kylian mbappe,NaN,NaN,ST,91,97,90,80,92,36,78,kylian,mbappe,k. mbappe,231747
1,2,3,erling haaland,NaN,NaN,ST,91,89,93,66,80,45,88,erling,haaland,e. haaland,239085
2,3,4,kevin de bruyne,NaN,NaN,CM,91,72,88,94,87,65,78,kevin,de bruyne,k. de bruyne,192985
3,5,6,lionel messi,NaN,NaN,CF,90,80,87,90,94,33,64,lionel,messi,l. messi,158023
4,7,8,karim benzema,NaN,NaN,CF,90,79,88,83,87,39,78,karim,benzema,k. benzema,165153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16041,17321,17318,deven sawhney,NaN,NaN,LB,47,62,26,34,42,43,57,deven,sawhney,d. sawhney,266248
16042,17322,17318,wang tengda,NaN,NaN,CM,47,61,37,45,48,43,47,wang,tengda,w. tengda,258959
16043,17323,17318,wu yuhang,NaN,NaN,CDM,47,56,34,43,45,43,51,wu,yuhang,w. yuhang,269541
16044,17324,17318,ye daoxin,NaN,NaN,CB,47,58,28,33,39,49,48,ye,daoxin,y. daoxin,275671
